**LOADING DEPENDENCIES** 

In [ ]:
pip install ffmpeg-python

In [ ]:
pip install av

In [ ]:
import os
import cv2
import torch
import random
import ffmpeg
import warnings
import numpy as np
import pandas as pd 
import soundfile as sf
from PIL import Image
from pydub import AudioSegment
from scipy.signal import resample 

**LOADING SPEECH TO TEXT MODEL** 

In [ ]:
!pip install --upgrade pip 

In [ ]:
!pip install --upgrade transformers datasets[audio] accelerate 

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline 

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device, 
    #generate_kwargs ={"language": "en", "forced_decoder_ids": None}      # uncomment this line to generate english-translated audio transcriptions 
)

**LOADING DATA** 

In [ ]:
# Extracting all required ids 

df = pd.read_csv('/kaggle/input/youtube-data/all_unique_codes3.csv') 
df = df[df['Available?'] == 'Yes'] 
df 

In [ ]:
# Extracting video ids and primary labels 

video_ids = list(df['Video link']) 
primary_labels = list(df['Primary Tag']) 

In [ ]:
# Extracting ids of ads downloaded to kaggle 

available_ad_ids = os.listdir('/kaggle/input/youtube-data/Ads/Ads') 
print(len(available_ad_ids)) 

**OBTAINING AUDIO TRANSCRIPTIONS** 

In [ ]:
len(video_ids) 

In [ ]:
ids = [] 
transcriptions = [] 

for i in range(len(video_ids)): 
    if video_ids[i] in available_ad_ids: 
        print(i)
        audio_path = '/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i] + '/audio.mp3' 
        response = pipe(audio_path, return_timestamps=True)
        transcriptions.append(response) 
        ids.append(video_ids[i]) 

In [ ]:
import pandas as pd 

df = pd.DataFrame({
    'Video Id': ids, 
    'Transcription': transcriptions 
})

df.head() 

In [ ]:
df.to_csv('/kaggle/working/native-language-transcriptions.csv', index=False) 